In [1]:
import sys
sys.path.append("/anaconda/envs/azureml_py38/lib/python3.8/site-packages")

In [2]:
import psycopg2
import pandas as pd
import numpy as np

In [4]:
def get_db_cursor():
    username = "doadmin"
    password = 'yzmodwh2oh16iks6'
    host = 'db-postgresql-cl1-do-user-2276924-0.db.ondigitalocean.com'
    port = 25060
    database = 'MandatoryMetadata'
    schema = 'historical'

    conn = psycopg2.connect(host=host, database=database,
                            user=username, password=password, port=port)
    cur = conn.cursor()
    cur.execute("SET search_path TO " + schema)

    return conn, cur

In [5]:
conn, cur = get_db_cursor()


In [5]:
v1_samples = pd.read_csv('./outputFiles/spectra.csv',usecols=[0])

In [6]:
v1_samples

,Unnamed: 0
0,AGD207-408SA0706
1,AGD207-431SA0001
2,AGD207-431SA0002
3,AGD207-431SA0003
4,AGD207-431SA0004
...,...
87362,CW017SA7164
87363,CW017SA7167
87364,CW017SA7168
87365,CW104SA0052


In [12]:
v2_samples = pd.read_sql('select DISTINCT sample_code from MandatoryMetadata', con = conn)

/tmp/ipykernel_4684/3980953645.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  v2_samples = pd.read_sql('select DISTINCT sample_code from MandatoryMetadata', con = conn)


In [13]:
v2_samples

,sample_code
0,CJ366SA0002
1,CC176SA2464
2,CE006SA2172
3,CE006SA1261
4,CC245SA0004
...,...
128766,CW133SA0089
128767,CC068SA0581
128768,CS095SA0106
128769,CS667SA0014


In [14]:
v2_samples = v2_samples.loc[~v2_samples['sample_code'].isin(v1_samples['Unnamed: 0'])]

In [15]:
v2_samples

,sample_code
0,CJ366SA0002
2,CE006SA2172
7,CK486SA0005
8,CA236SA0323
11,CF177SA3204
...,...
128765,CZ030SA0042
128766,CW133SA0089
128768,CS095SA0106
128769,CS667SA0014


In [16]:
sample_codes = v2_samples['sample_code'].values

In [17]:
if(len(sample_codes) < 5000):
    count  = len(sample_codes)
    step=count
elif(len(sample_codes) < 70000):
    count = len(sample_codes)
    step=5000
else:
    count = 70000
    step=5000
start = 0


In [18]:
count

57352

In [19]:
start

0

In [20]:
step

5000

In [21]:
spectra = pd.DataFrame()

In [22]:
def convertSpectra(df):
    print("Converting spectra")
    df_ = pd.DataFrame([i[[i for i in i.keys()][0]] for i in df['averaged_spectra'].values],columns = np.arange(522,3977,2))
    df_.index = df.index
    print("Spectra converted")
    return df_

In [23]:
for i in np.arange(start, count, step):
        
    print("Fetching spectra from {}".format(start))
    samples = [i for i in sample_codes][start:start+step]
    print(len(samples))
    query = f"SELECT DISTINCT mandatorymetadata.sample_code, spectraldata.metadata_id, spectraldata.averaged_spectra FROM spectraldata INNER JOIN mandatorymetadata ON mandatorymetadata.metadata_id = spectraldata.metadata_id WHERE is_finalized=True AND passed=True AND is_active=True AND averaged=True AND sample_code IN {str(samples).replace('[','(').replace(']',')')}"

    _ = pd.read_sql(query, con=conn)
    print("This is the spectra",_)
    spectra = pd.concat([spectra, _], axis=0)
    start = start + step
    if (count-step) > 5000:
        step=5000
    else:
        step = count-step





Fetching spectra from 0
5000
This is the spectra            sample_code  metadata_id  \
0      AGD103-40SA0001         9304   
1     AGD207-408SA0010         8409   
2     AGD207-408SA0050         9826   
3     AGD207-408SA0128         8414   
4     AGD207-408SA0169         9825   
...                ...          ...   
4278   I48-1-103SA0001         8559   
4279            ISRAEL         4850   
4280       TMA-FL-0534         8667   
4281       TMA-FL-0535        10549   
4282       TMA-FL-0599        10569   

                                       averaged_spectra  
0     {'522': [2.2946243572164216, 2.252323720423919...  
1     {'522': [2.0243299051640333, 2.046907200185822...  
2     {'522': [2.0714313250843235, 1.973399886588285...  
3     {'522': [2.335163074422063, 2.3173864933474606...  
4     {'522': [2.2009018853383973, 2.196002606635404...  
...                                                 ...  
4278  {'522': [1.8737280792526008, 1.844528032381382...  
4279  {'522': [2.6

/tmp/ipykernel_4684/1185931234.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
/tmp/ipykernel_4684/1185931234.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
/tmp/ipykernel_4684/1185931234.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
/tmp/ipykernel_4684/1185931234.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 o

In [24]:
spectra = spectra.set_index("sample_code")

In [25]:
conn.close()


In [52]:
spectra = spectra.set_index('sample_code')


In [26]:
spectra = convertSpectra(spectra)


spectra.to_csv("./outputFiles/spectraldata.csv")

Converting spectra
Spectra converted


In [21]:
spectra

,522,524,526,528,530,532,534,536,538,540,...,3958,3960,3962,3964,3966,3968,3970,3972,3974,3976
0,2.364527,2.380267,2.415282,2.431268,2.441101,2.462991,2.473978,2.428879,2.422211,2.447353,...,1.170164,1.170703,1.170675,1.170313,1.170076,1.169900,1.169492,1.168838,1.168238,1.167781
1,2.324680,2.324753,2.373763,2.411027,2.402217,2.347667,2.317289,2.338107,2.375101,2.368011,...,1.149651,1.149604,1.149459,1.148917,1.148238,1.147781,1.147594,1.147623,1.147640,1.147664
2,2.397271,2.428240,2.399225,2.358932,2.332269,2.308703,2.294938,2.292284,2.298963,2.309691,...,1.160591,1.160070,1.159524,1.159550,1.159486,1.158533,1.157588,1.157537,1.157760,1.157822
3,2.394315,2.391048,2.357433,2.361828,2.380389,2.410625,2.426476,2.401460,2.370650,2.361235,...,1.152852,1.152934,1.152444,1.151799,1.151322,1.150696,1.150411,1.150530,1.150366,1.150150
4,2.381931,2.383940,2.376204,2.340311,2.348089,2.348603,2.345284,2.341323,2.328165,2.294485,...,1.159744,1.158565,1.157161,1.156499,1.156687,1.156656,1.155951,1.155213,1.154969,1.155184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5709,2.431712,2.491956,2.525230,2.497294,2.461680,2.457347,2.479673,2.508331,2.492839,2.441814,...,1.129957,1.129489,1.128628,1.128164,1.128154,1.128244,1.128160,1.127920,1.127710,1.127501
5710,2.383299,2.418019,2.405626,2.382745,2.379580,2.403409,2.391373,2.320085,2.273765,2.275257,...,1.149114,1.149265,1.149298,1.148614,1.147901,1.148023,1.148273,1.148000,1.147757,1.147768
5711,2.388890,2.350033,2.309996,2.327226,2.352492,2.330514,2.316581,2.350303,2.387681,2.412264,...,1.171154,1.170969,1.170589,1.170483,1.170664,1.170294,1.169510,1.169155,1.169263,1.169555
5712,2.477091,2.414741,2.403005,2.449703,2.481381,2.444921,2.382814,2.372414,2.406186,2.411119,...,1.161688,1.161186,1.160171,1.159406,1.159063,1.158751,1.158270,1.157872,1.157729,1.157586


In [22]:
import pandas as pd
spectra= pd.read_csv("outputFiles/spectraldata.csv",usecols=[0])
len(spectra)

123300

In [27]:
spectra

,522,524,526,528,530,532,534,536,538,540,...,3958,3960,3962,3964,3966,3968,3970,3972,3974,3976
sample_code,,,,,,,,,,,,,,,,,,,,,
AGD103-40SA0001,2.294624,2.252324,2.252763,2.264666,2.255082,2.239374,2.223874,2.219874,2.212979,2.177654,...,1.422320,1.420521,1.420258,1.421809,1.422426,1.420608,1.418333,1.417951,1.418956,1.419416
AGD207-408SA0010,2.024330,2.046907,2.043177,2.033848,2.034393,2.073468,2.145949,2.173295,2.110662,2.067651,...,1.432352,1.431705,1.432530,1.431856,1.428591,1.427078,1.427711,1.426269,1.422301,1.421177
AGD207-408SA0050,2.071431,1.973400,1.905598,1.892876,1.889285,1.885217,1.911854,1.932437,1.902481,1.876775,...,1.372569,1.372696,1.375012,1.375759,1.376213,1.379223,1.379211,1.373168,1.366714,1.362238
AGD207-408SA0128,2.335163,2.317386,2.324318,2.322120,2.296425,2.284984,2.278755,2.277955,2.282519,2.271201,...,1.332006,1.330292,1.329286,1.329465,1.329426,1.327830,1.325649,1.324013,1.323073,1.323118
AGD207-408SA0169,2.200902,2.196003,2.226497,2.232089,2.222513,2.216196,2.197458,2.171872,2.161863,2.167995,...,1.370889,1.370175,1.370109,1.370543,1.370074,1.368265,1.366876,1.366383,1.365442,1.364319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FA944SA0035,2.112599,2.088368,2.095636,2.091958,2.078875,2.041343,1.988712,1.969161,1.989761,1.993117,...,1.258947,1.258221,1.257999,1.257660,1.256922,1.256442,1.256209,1.255749,1.255180,1.254776
FA944SA0036,2.115692,2.129940,2.103012,2.075006,2.072764,2.080007,2.075533,2.053104,2.030145,2.018534,...,1.281532,1.281899,1.282251,1.281950,1.281488,1.280942,1.279549,1.277748,1.276897,1.277108
FA944SA0037,2.292454,2.322450,2.332670,2.325429,2.323228,2.320085,2.332086,2.352944,2.333360,2.300843,...,1.345511,1.346190,1.346026,1.344989,1.344079,1.343586,1.342809,1.341466,1.340403,1.339693


In [27]:
start = 0 
stop = 5000
for n in np.arange(start, (19000), 5000):
    print(n)

0
5000
10000
15000


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('outputFiles/PCC_Classes/aluminium.csv')

In [6]:
preds = pd.read_csv("outputFiles/predictions/DLv2.2/aluminium_preds.csv")

In [7]:
preds

,sample_code,0
0,ML-SS21-1,845.05840
1,ML-SS21-1,930.48130
2,ML-SS21-1,845.21277
3,ML-SS21-1,888.77905
4,ML-SS21-1,907.39136
...,...,...
123302,ML-SS21-3,846.77010
123303,ML-SS21-3,893.17316
123304,ML-SS21-3,921.44040
123305,ML-SS21-3,873.87006


In [8]:
df

,Unnamed: 0,sample_code,aluminium,0,Difference,Actual_PCC,Predicted_PCC,PCC_Class
0,1489,CC176SA5654,1040.0,884.54590,-155.45410,4.0,NaN,NaN
1,1497,CC176SA5627,1120.0,980.52155,-139.47845,4.0,NaN,NaN
2,1523,CC176SA5628,925.0,560.16120,-364.83880,3.0,NaN,NaN
3,1536,CC176SA5652,859.0,871.38885,12.38885,3.0,3.0,0.0
4,1562,CC176SA5641,711.0,763.31380,52.31380,3.0,3.0,0.0
...,...,...,...,...,...,...,...,...
7233,93208,CK002SA1803,1100.0,1066.29250,-33.70750,4.0,4.0,0.0
7234,93220,CK002SA1806,1030.0,1117.54700,87.54700,4.0,4.0,0.0
7235,93393,CK002SA1801,1000.0,1007.66600,7.66600,3.0,3.0,0.0
7236,94931,CJ030SA5678,995.0,1105.27750,110.27750,3.0,4.0,1.0


In [10]:
comb = pd.merge(df, preds, on='sample_code', how='inner')

In [11]:
comb.loc[comb['PCC_Class'] <= 1]

,Unnamed: 0,sample_code,aluminium,0_x,Difference,Actual_PCC,Predicted_PCC,PCC_Class,0_y
3,1536,CC176SA5652,859.0,871.38885,12.38885,3.0,3.0,0.0,871.38885
4,1562,CC176SA5641,711.0,763.31380,52.31380,3.0,3.0,0.0,763.31380
5,1583,CC176SA5645,520.0,640.46515,120.46515,2.0,2.0,0.0,640.46515
6,1618,CC176SA5647,945.0,863.47784,-81.52216,3.0,3.0,0.0,863.47784
8,1652,CC176SA5650,517.0,572.62830,55.62830,2.0,2.0,0.0,572.62830
...,...,...,...,...,...,...,...,...,...
53456,93206,CK002SA1805,835.0,1046.02430,211.02430,3.0,4.0,1.0,1046.02430
53457,93208,CK002SA1803,1100.0,1066.29250,-33.70750,4.0,4.0,0.0,1066.29250
53458,93220,CK002SA1806,1030.0,1117.54700,87.54700,4.0,4.0,0.0,1117.54700
53459,93393,CK002SA1801,1000.0,1007.66600,7.66600,3.0,3.0,0.0,1007.66600


In [18]:
spectra = pd.read_csv('outputFiles/spectraldata.csv', index_col=0, engine='c')

In [20]:
spectra.loc[spectra.index.isin(comb.loc[comb['PCC_Class'] <= 1]['sample_code'])]

,522,524,526,528,530,532,534,536,538,540,...,3958,3960,3962,3964,3966,3968,3970,3972,3974,3976
sample_code,,,,,,,,,,,,,,,,,,,,,
CC176SA5652,2.154246,2.092609,2.077228,2.101096,2.120622,2.123532,2.127089,2.126469,2.119017,2.109390,...,1.255574,1.254978,1.253972,1.253552,1.253228,1.252433,1.251589,1.250707,1.249439,1.248191
CC176SA5641,2.239911,2.182096,2.130445,2.081945,2.047989,2.055831,2.099324,2.130741,2.105824,2.057918,...,1.132173,1.131954,1.131708,1.131565,1.131205,1.130505,1.129671,1.128951,1.128448,1.128007
CC176SA5645,2.093391,2.111062,2.063969,2.010349,2.003698,2.011247,1.988045,1.952734,1.942120,1.955275,...,1.161763,1.161122,1.160494,1.160462,1.160539,1.159970,1.159012,1.158122,1.157647,1.157516
CC176SA5647,2.042394,2.090577,2.111565,2.067445,2.036329,2.053858,2.076758,2.071446,2.061716,2.063592,...,1.130418,1.129733,1.128900,1.128374,1.128039,1.127651,1.127224,1.126863,1.126352,1.125385
CC176SA5650,2.270940,2.229641,2.189799,2.145278,2.138737,2.172354,2.205997,2.246785,2.261879,2.219524,...,1.179193,1.178446,1.177620,1.176593,1.175882,1.175641,1.175558,1.175543,1.175335,1.174862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CK002SA1805,2.275789,2.255795,2.238824,2.242241,2.230048,2.199329,2.175204,2.179353,2.180641,2.145780,...,1.286755,1.285515,1.284647,1.284983,1.286087,1.286118,1.284773,1.283282,1.282345,1.281821
CK002SA1803,2.521574,2.443199,2.414373,2.409841,2.420043,2.455131,2.472920,2.451894,2.434731,2.430271,...,1.325941,1.324451,1.324035,1.324074,1.323478,1.322610,1.322125,1.321414,1.320255,1.318866
CK002SA1806,2.634936,2.540841,2.504133,2.537708,2.569144,2.512635,2.449713,2.435081,2.456787,2.475545,...,1.315967,1.315506,1.315423,1.314387,1.312685,1.312006,1.311860,1.311262,1.310908,1.310861


In [16]:
spectra.loc[spectra.index.isin(preds['sample_code'])]

,522,524,526,528,530,532,534,536,538,540,...,3958,3960,3962,3964,3966,3968,3970,3972,3974,3976


In [17]:
spectra

,522,524,526,528,530,532,534,536,538,540,...,3958,3960,3962,3964,3966,3968,3970,3972,3974,3976
AGD207-408SA0706,2.096135,2.074343,2.077232,2.100555,2.109029,2.113436,2.103539,2.076615,2.058088,2.052834,...,1.397774,1.396845,1.396773,1.396401,1.395748,1.395857,1.395875,1.394762,1.394079,1.393831
AGD207-431SA0001,2.154667,2.128691,2.090363,2.061056,2.036317,2.019021,2.018069,2.030793,2.041479,2.035714,...,1.319761,1.319318,1.319803,1.319366,1.317630,1.316362,1.315863,1.315292,1.314809,1.314864
AGD207-431SA0002,2.412217,2.413204,2.360303,2.307860,2.296765,2.316080,2.321781,2.294391,2.270676,2.284707,...,1.306064,1.306364,1.306144,1.305445,1.305173,1.305272,1.304674,1.303443,1.302389,1.301506
AGD207-431SA0003,2.059137,2.093001,2.136247,2.157497,2.152817,2.144714,2.131581,2.108572,2.105603,2.127556,...,1.306060,1.305862,1.305825,1.305172,1.304238,1.303409,1.302201,1.300902,1.300176,1.300167
AGD207-431SA0004,2.279650,2.237250,2.218038,2.231755,2.238702,2.222442,2.208411,2.211492,2.218197,2.235896,...,1.184210,1.183787,1.183616,1.183282,1.182663,1.182083,1.181676,1.181184,1.180578,1.180207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CW017SA7164,2.187465,2.130583,2.079146,2.053846,2.028107,1.993453,1.961532,1.942093,1.952680,1.984957,...,1.150593,1.149948,1.150184,1.150910,1.151015,1.150748,1.150615,1.150541,1.150394,1.149878
CW017SA7167,2.154086,2.122482,2.061072,2.017800,2.007365,2.026743,2.058742,2.077140,2.069497,2.065080,...,1.226464,1.225381,1.224707,1.224440,1.224299,1.224463,1.225003,1.225329,1.224778,1.223832
CW017SA7168,2.159694,2.118398,2.149818,2.189918,2.170442,2.141771,2.140313,2.135926,2.115328,2.103800,...,1.188755,1.188114,1.188517,1.189058,1.188850,1.188428,1.187938,1.187309,1.186815,1.186573
CW104SA0052,1.915089,1.907579,2.006204,2.047937,2.043367,2.039258,2.045996,2.070376,2.097082,2.112224,...,1.735016,1.731109,1.727900,1.728896,1.729304,1.726673,1.724738,1.723750,1.723889,1.726787


In [22]:
s = pd.read_csv('outputFiles/PCC1/aluminium.csv', engine='c', index_col=0)

Bad pipe message: %s [b'\xfa&\x07\xcd\xf2\xba\xaeB\xd7WY\xdfM25)a\xc2 \xe3\t\x8c\x85\xbb\xcc{\x0eL\xd0\xa0\xf6q{\xc2\xbeM\xd7H\xbc\xc6r[\xf1\xc9\xe2\x1ai\x9e\x03\xcd\x87\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06', b'\x07\x08']
Bad pipe message: %s [b'\t\x08\n\x08\x0b\x08\x04']
Bad pipe message: %s [b'\x08\x06\x04\x01\x05\x01\x06', b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 t\xf1\xe9[\xf0\xd9}|\x9eV|\xbeV#\xfc\xc3\xc8`f(b?']
Bad pipe message: %s [b'\xdc\xd2\xde\\ZLu9]\xa4+!\xcb\xa7qp\xaft\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc

In [23]:
arr = [1,2,3,4,5,6,7]

In [25]:
result = [sum(arr[0:i]) for i in range(len(arr))]



In [26]:
result

[0, 1, 3, 6, 10, 15, 21]